# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tazacorte,28.6290,-17.9293,22.54,93,0,2.06,ES,1698730636
1,1,waitangi,-43.9535,-176.5597,13.23,88,100,6.71,NZ,1698730636
2,2,bethel,41.3712,-73.4140,8.93,77,20,1.54,US,1698730637
3,3,kerikeri,-35.2268,173.9474,19.95,87,57,0.89,NZ,1698730637
4,4,turinsk,58.0458,63.6960,-4.12,84,99,1.83,RU,1698730637


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [16]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df[city_data_df["Max Temp"]<=27]


# Drop any rows with null values
ideal_city_df = ideal_city_df.dropna(how='any')

ideal_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tazacorte,28.6290,-17.9293,22.54,93,0,2.06,ES,1698730636
1,1,waitangi,-43.9535,-176.5597,13.23,88,100,6.71,NZ,1698730636
2,2,bethel,41.3712,-73.4140,8.93,77,20,1.54,US,1698730637
3,3,kerikeri,-35.2268,173.9474,19.95,87,57,0.89,NZ,1698730637
4,4,turinsk,58.0458,63.6960,-4.12,84,99,1.83,RU,1698730637
...,...,...,...,...,...,...,...,...,...,...
568,568,chifeng,42.2683,118.9636,17.78,22,38,6.00,CN,1698730791
570,570,kodinsk,58.6881,99.1844,-9.76,41,2,2.38,RU,1698730792
571,571,orzysz,53.8097,21.9481,11.73,86,100,6.11,PL,1698730792
572,572,akcakoca,41.0866,31.1162,18.37,88,51,2.47,TR,1698730792


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

hotel_df 

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,tazacorte,ES,28.6290,-17.9293,93,
1,waitangi,NZ,-43.9535,-176.5597,88,
2,bethel,US,41.3712,-73.4140,77,
3,kerikeri,NZ,-35.2268,173.9474,87,
4,turinsk,RU,58.0458,63.6960,84,
...,...,...,...,...,...,...
572,akcakoca,TR,41.0866,31.1162,88,
573,magadi,IN,12.9667,77.2333,70,
574,perry,US,37.2506,-83.1999,92,
575,katima mulilo,NaN,-17.5000,24.2667,58,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
limit = 20


params = {
    "categories":categories,
    "limit":limit,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tazacorte - nearest hotel: App Leyma
waitangi - nearest hotel: Hotel Chathams
bethel - nearest hotel: Hampton Inn Danbury
kerikeri - nearest hotel: Avalon Resort
turinsk - nearest hotel: Скорпион
timbuktu - nearest hotel: Hotel La Maison
luderitz - nearest hotel: Seaview Hotel Zum Sperrgebiet
thompson - nearest hotel: Thompson Inn
tocopilla - nearest hotel: hotel Isidora
penrith - nearest hotel: Villa Bianca
nova sintra - nearest hotel: Residência Ka Dencho
tiksi - nearest hotel: Арктика
alesund - nearest hotel: Brosundet
mount pearl - nearest hotel: Sandman Signature St. John's
hawaiian paradise park - nearest hotel: No hotel found
port-aux-francais - nearest hotel: Keravel
kashgar - nearest hotel: 深航国际酒店
talnakh - nearest hotel: Талнах
adamstown - nearest hotel: No hotel found
manismata - nearest hotel: No hotel found
huron - nearest hotel: No hotel found
danane - nearest hotel: Les Frontieres
recani - nearest hotel: Hotel Merzouga
shubarshi - nearest hotel: No 

kapa'a - nearest hotel: Pono Kai Resort
new norfolk - nearest hotel: Woodbridge on the Derwent
extrema - nearest hotel: Hotel San Marco
ashland - nearest hotel: Quality Inn
al jawf - nearest hotel: No hotel found
kangal - nearest hotel: No hotel found
taltal - nearest hotel: Hostería Taltal
tot'ma - nearest hotel: No hotel found
kavaratti - nearest hotel: No hotel found
vershino-darasunskiy - nearest hotel: No hotel found
nar'yan-mar - nearest hotel: Гостиница "Агат"
chibougamau - nearest hotel: No hotel found
smithers - nearest hotel: Sunshine Inn Hotel
simanggang - nearest hotel: Mega Inn
upington - nearest hotel: Die Eiland Holiday Ressort
chonchi - nearest hotel: Hotel Huildín
veinticinco de mayo - nearest hotel: Jorly
georgetown - nearest hotel: Page 63 hostel
lyubinskiy - nearest hotel: No hotel found
juneau - nearest hotel: Ramada by Wyndham Juneau
reggane - nearest hotel: No hotel found
bemidji - nearest hotel: Hampton Inn and Suites
zhangzhou - nearest hotel: 中国女排公寓
petrozavod

peniche - nearest hotel: Residencial Maciel
pimentel - nearest hotel: Hospedaje Calle del Sol
borgarnes - nearest hotel: Borgarnes
weno - nearest hotel: High Tide Hotel
tinogasta - nearest hotel: No hotel found
rawson - nearest hotel: Hotel Deportivo
callao - nearest hotel: Casa Ronald
benguela - nearest hotel: Hotel Moibela
kalmunai - nearest hotel: Saji new food
palikir - national government center - nearest hotel: Mangrove Bay Hotel
armacao de buzios - nearest hotel: Vila da Santa
kayyerkan - nearest hotel: No hotel found
urangan - nearest hotel: Ramada
pandan - nearest hotel: No hotel found
lagos - nearest hotel: No hotel found
pisco - nearest hotel: La Portada
guerrero negro - nearest hotel: Plaza sal paraiso
aoulef - nearest hotel: دار الضياف
bestwig - nearest hotel: Hengsbach
yongchuan - nearest hotel: 時代天街
carutapera - nearest hotel: No hotel found
al qaryatayn - nearest hotel: No hotel found
fada - nearest hotel: No hotel found
malanje - nearest hotel: Hotel PORTVGALIA
coracor

,City,Country,Lat,Lng,Humidity,Hotel Name
0,tazacorte,ES,28.6290,-17.9293,93,App Leyma
1,waitangi,NZ,-43.9535,-176.5597,88,Hotel Chathams
2,bethel,US,41.3712,-73.4140,77,Hampton Inn Danbury
3,kerikeri,NZ,-35.2268,173.9474,87,Avalon Resort
4,turinsk,RU,58.0458,63.6960,84,Скорпион
...,...,...,...,...,...,...
572,akcakoca,TR,41.0866,31.1162,88,아착코자숙소
573,magadi,IN,12.9667,77.2333,70,Udupi Upahara
574,perry,US,37.2506,-83.1999,92,Hampton Inn & Suites
575,katima mulilo,NaN,-17.5000,24.2667,58,Zambezi River Lodge


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

# Display the map
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)